In [1]:
# internal imports
from LM import LMBackend
from TTS import TTSBackend
from SpeechRecognition import SpeechRecognitionBackend
# external imports
from threading import Thread
import queue
import sounddevice as sd
import numpy as np
import time

In [2]:
# Load Backend Classes
lm_instance = LMBackend()
lm_instance.init("/Users/lipeihong/Desktop/IT Project/py3/Language_Model/LM/gemma-2-2b-it")


[+] initializing LMBackend
    -> Using device:  mps


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

    -> LMBackend loaded


In [3]:
tts_instance = TTSBackend()
tts_instance.init("espnet/fastspeech2_conformer")


[+] initializing TTS System
    -> Using device:  mps


Some weights of FastSpeech2ConformerHifiGan were not initialized from the model checkpoint at espnet/fastspeech2_conformer_hifigan and are newly initialized: ['mean', 'scale']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    -> TTS System loaded


In [4]:
sr_instance = SpeechRecognitionBackend()
sr_instance.init("/Users/lipeihong/Downloads/whisper-small.en")

[+] initializing SpeechRecognitionBackend
    -> Using device:  mps
    -> SpeechRecognitionBackend loaded


In [5]:
# Global Variables
audio_queue = queue.Queue()
# Constants
RECORD_THRESHOLD = 200
SAMPLING_RATE = 16000
REC_DURATION = 1

In [6]:
# Function definitions
""" These functions are from Speech-Recognition branch, with process_audio_stream modified to utlise all backends to generate voice response """
""" Different ways to coordinate these functions. """

def audio_callback(indata):
    audio_queue.put(indata.copy())  # Put the captured audio

def get_mic_amplitude(input_stream, duration):
    data, overflowed = input_stream.read(SAMPLING_RATE * duration)
    return np.linalg.norm(data) * 10

# Code to normalize text(removing non-utf8 character generated by LM)
def remove_non_utf8(text):
    # Encode the string to bytes, ignoring any non-UTF-8 characters
    encoded_text = text.encode("utf-8", "ignore")
    # Decode it back to a string
    decoded_text = encoded_text.decode("utf-8")
    return decoded_text

def process_audio_stream(audio_input:dict):
    """ from voice input to voice response """
    # recognize audio
    sr_result = sr_instance.recognize(audio_input)
    print("**** Debug ****: ", sr_result)
    # generate response
    lm_result = lm_instance.generate_text(sr_result["text"])
    
    # generate voice response
    tts_result = tts_instance.synthesize(remove_non_utf8(lm_result))
    
    # play audio
    sd.play(tts_result["array"], samplerate=tts_result["sampling_rate"])
    sd.wait()



In [7]:
def debug_player(audio_data):
    """Debug function to play the audio from the queue."""
    print("Playing audio...")
    sd.play(audio_data, SAMPLING_RATE)

# Main function
def main():
    voice_input_stream = sd.InputStream(channels=1, samplerate=SAMPLING_RATE)
    voice_input_stream.start()
    
    sound_amp_queue = queue.Queue()
    can_record:bool = False
    
    try:
        while True:
            # detect sound amplitude to determine if we should record
            if (sound_amp_queue.qsize() > 15):
                sound_amp_queue.get()
            
            data, overflowed = voice_input_stream.read(SAMPLING_RATE * REC_DURATION)
            volume_norm = np.linalg.norm(data) * 10
            
            if sound_amp_queue.qsize() < 3:
                sound_amp_queue.put(volume_norm)
                continue
            
            # print("amplitude queue: ", sound_amp_queue.queue)
            avg_mic_amplitude = sum(sound_amp_queue.queue) / sound_amp_queue.qsize()
            # only collect background noise level, not outliers.
            if abs(volume_norm - avg_mic_amplitude) > RECORD_THRESHOLD:
                can_record = True
            else:
                sound_amp_queue.put(volume_norm)
            
            if can_record:
                print("[+] Recording...")
                audio_array = np.empty((0, 1)) 
                record_amp_queue = queue.Queue()
                while can_record:
                    audio_array = np.append(audio_array, data)
                    data, overflowed = voice_input_stream.read(SAMPLING_RATE * REC_DURATION)
                    rec_volume_norm = np.linalg.norm(data) * 10
                    # determine when to stop recording
                    record_amp_queue.put(rec_volume_norm)
                    if record_amp_queue.qsize() > 3:
                        record_amp = sum(record_amp_queue.queue) / record_amp_queue.qsize()
                        # terminate recording if the amplitude back to normal
                        if abs(avg_mic_amplitude - record_amp) < RECORD_THRESHOLD:
                            can_record = False
                            audio_data = {"array": audio_array, "sampling_rate": SAMPLING_RATE}
                            # process audio stream
                            # debug_player(audio_data["array"])
                            process_audio_stream(audio_data.copy())
                            print("Recording stopped.")
                            break
                        record_amp_queue.get()
    finally:
        voice_input_stream.stop()
        voice_input_stream.close()

In [8]:
main()

[+] Recording...


/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/transformers/models/whisper/generation_whisper.py:483: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


**** Debug ****:  {'text': " Hello, what's your name?"}
Recording stopped.
[+] Recording...
**** Debug ****:  {'text': " It's nice to meet you."}
Recording stopped.
[+] Recording...
**** Debug ****:  {'text': ' My name is Ike.'}
Recording stopped.


KeyboardInterrupt: 